# Bibliotecas

In [98]:
%matplotlib inline

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import math

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, HuberRegressor, Ridge, RidgeCV, ElasticNet
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

# Funções Auxiliares

In [52]:
def percentual_outlier(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    minimum = Q1 - (1.5 * IQR)
    maximum = Q3 + (1.5 * IQR)
    num_outliers =  np.sum((data < minimum) | (data > maximum))
    num_total = data.count()
    return (num_outliers/num_total)*100

# Carregando os Dados

In [3]:
df = pd.read_csv("./dados_regressao.csv")

In [4]:
df.head()

,numero_solicitacao,razaoSocial,nomeFantasia,cnpjSemTraco,maiorAtraso,margemBrutaAcumulada,percentualProtestos,primeiraCompra,prazoMedioRecebimentoVendas,titulosEmAberto,...,periodoDemonstrativoEmMeses,custos,anoFundacao,intervaloFundacao,capitalSocial,restricoes,empresa_MeEppMei,scorePontualidade,limiteEmpresaAnaliseCredito,dataAprovadoNivelAnalista
0,1,James Richardson-Patel,Alexandra Williams,KEBE17609492220843,0,0.252448,0.0,2015-12-10T00:00:00,0,0.00,...,12.0,922289.0,2003.0,Acima de 17 anos,90000.0,False,True,1.0,72600.0,2020-02-03T20:57:33
1,3,Joanna Hudson,Dr. David Rees,GCVQ28531614261293,4,0.624777,0.0,2019-11-27T00:00:00,0,0.00,...,12.0,1485136.0,2014.0,De 6 a 10 anos,20000.0,False,False,1.0,5760.0,2020-02-04T16:40:49
2,5,Nigel Lee,Dr. Stanley Duncan,CGQN15826802440348,20,0.454088,0.0,2010-07-13T00:00:00,20,1486.95,...,9.0,40680051.0,2002.0,Acima de 17 anos,75000.0,False,False,1.0,89000.0,2020-02-04T15:06:28
3,7,Alexander Baker-Wells,Dr. Caroline Ali,PYOL43118620147076,16,0.000000,NaN,2011-03-02T11:27:13,0,0.00,...,12.0,45795943.0,2004.0,De 11 a 16 anos,30515674.0,True,False,0.0,20635703.0,2020-02-05T14:31:22
4,8,Jean Berry,Gregory Gould,NLUX93496216176463,4,0.350175,0.0,2007-07-02T00:00:00,102,208980.00,...,9.0,0.0,2000.0,Acima de 17 anos,120000.0,False,False,1.0,768233.0,2020-02-05T20:05:40


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5095 entries, 0 to 5094
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   numero_solicitacao           5095 non-null   int64  
 1   razaoSocial                  5095 non-null   object 
 2   nomeFantasia                 5095 non-null   object 
 3   cnpjSemTraco                 5095 non-null   object 
 4   maiorAtraso                  5095 non-null   int64  
 5   margemBrutaAcumulada         5095 non-null   float64
 6   percentualProtestos          4300 non-null   float64
 7   primeiraCompra               5061 non-null   object 
 8   prazoMedioRecebimentoVendas  5095 non-null   int64  
 9   titulosEmAberto              5095 non-null   float64
 10  valorSolicitado              5095 non-null   float64
 11  status                       5095 non-null   object 
 12  definicaoRisco               5095 non-null   object 
 13  diferencaPercentua

In [115]:
df_num = df.drop(['cnpjSemTraco', 'percentualProtestos', 'dataAprovadoEmComite', 'dataAprovadoNivelAnalista', 'numero_solicitacao', 'razaoSocial', 'nomeFantasia', 'anoFundacao', 'status', 'definicaoRisco', 'diferencaPercentualRisco', 'dashboardCorrelacao',
         'periodoBalanco', 'primeiraCompra', 'intervaloFundacao', 'periodoDemonstrativoEmMeses', 'restricoes', 'empresa_MeEppMei'], axis=1)

In [116]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5095 entries, 0 to 5094
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   maiorAtraso                  5095 non-null   int64  
 1   margemBrutaAcumulada         5095 non-null   float64
 2   prazoMedioRecebimentoVendas  5095 non-null   int64  
 3   titulosEmAberto              5095 non-null   float64
 4   valorSolicitado              5095 non-null   float64
 5   percentualRisco              5095 non-null   float64
 6   valorAprovado                5095 non-null   float64
 7   ativoCirculante              5095 non-null   float64
 8   passivoCirculante            5095 non-null   float64
 9   totalAtivo                   5095 non-null   float64
 10  totalPatrimonioLiquido       5095 non-null   float64
 11  endividamento                5095 non-null   float64
 12  duplicatasAReceber           5095 non-null   float64
 13  estoque           

In [117]:
for coluna in df_num.columns:
    data = df_num[coluna]
    perc = str(round(percentual_outlier(data), 2))
    print(f'Outliers na coluna {coluna} : {perc}%')

Outliers na coluna maiorAtraso : 11.27%
Outliers na coluna margemBrutaAcumulada : 0.12%
Outliers na coluna prazoMedioRecebimentoVendas : 5.99%
Outliers na coluna titulosEmAberto : 16.13%
Outliers na coluna valorSolicitado : 12.44%
Outliers na coluna percentualRisco : 0.47%
Outliers na coluna valorAprovado : 13.09%
Outliers na coluna ativoCirculante : 12.97%
Outliers na coluna passivoCirculante : 14.45%
Outliers na coluna totalAtivo : 13.29%
Outliers na coluna totalPatrimonioLiquido : 13.82%
Outliers na coluna endividamento : 17.98%
Outliers na coluna duplicatasAReceber : 12.86%
Outliers na coluna estoque : 13.54%
Outliers na coluna faturamentoBruto : 13.37%
Outliers na coluna margemBruta : 13.31%
Outliers na coluna custos : 13.6%
Outliers na coluna capitalSocial : 14.66%
Outliers na coluna scorePontualidade : 19.39%
Outliers na coluna limiteEmpresaAnaliseCredito : 16.7%


In [118]:
for column in df_num.columns:
    data = df_num[column]
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    minimum = Q1 - (1.5 * IQR)
    maximum = Q3 + (1.5 * IQR)
 
    outliers = ((data < minimum) | (data > maximum))
    df_num[column].loc[outliers] = np.nan
    
df_num.dropna(inplace=True)    

In [119]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2194 entries, 0 to 5093
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   maiorAtraso                  2194 non-null   float64
 1   margemBrutaAcumulada         2194 non-null   float64
 2   prazoMedioRecebimentoVendas  2194 non-null   float64
 3   titulosEmAberto              2194 non-null   float64
 4   valorSolicitado              2194 non-null   float64
 5   percentualRisco              2194 non-null   float64
 6   valorAprovado                2194 non-null   float64
 7   ativoCirculante              2194 non-null   float64
 8   passivoCirculante            2194 non-null   float64
 9   totalAtivo                   2194 non-null   float64
 10  totalPatrimonioLiquido       2194 non-null   float64
 11  endividamento                2194 non-null   float64
 12  duplicatasAReceber           2194 non-null   float64
 13  estoque           

In [135]:
x = df_num.drop('valorAprovado', axis=1)
y = df_num.valorAprovado

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [136]:
lm = LinearRegression()
linear = lm.fit(x_train, y_train)
y_pred = lm.predict(x_test)
linear.score(x_test, y_test)

0.7468735061927148

In [122]:
print('RMSE (root-mean-squared error): {}'.format(math.sqrt(mean_squared_error(y_test, y_pred))))

RMSE (root-mean-squared error): 26192.79821989001


In [124]:
sgd = SGDRegressor(max_iter=1000)
lin_sgd = sgd.fit(x_train, y_train)
lin_sgd.score(x_test, y_test)


-2.440008472566848e+35

In [126]:
ridge = Ridge()
lin_rdg = ridge.fit(x_train, y_train)
lin_rdg.score(x_test, y_test)

0.7472028491436262

In [127]:
ridgecv = RidgeCV()
lin_rdgcv = ridgecv.fit(x_train, y_train)
lin_rdgcv.score(x_test, y_test)

-1164.8955255844369

In [129]:
elastic = ElasticNet()
lin_elastic = elastic.fit(x_train, y_train)
lin_elastic.score(x_test, y_test)

0.7431920830638917